# 4. Generatory

to "funkcje", które przechowują swój stan pomiędzy wywołaniami. Generatory możemy wywoływać tylko przy pomocy metody "next".

In [9]:
# generation function
def simple_gen():
    a = 1
    yield a
    a += 1
    yield a
    
# generator
gen = simple_gen()

# first call
print next(gen)
# second call
print next(gen)

1
2


In [4]:
print type(simple_gen)
print type(gen)

<type 'function'>
<type 'generator'>


Generatory to funkcje, które są jednocześnie iteratorami i spełniają tzw. iterator protocol.

1. zwracają siebie w przypadku przekazania ich do funkcji "iter"
2. można na nich wywołać funkcję next
3. wywołują wyjątek StopIteration, gdy kończą swoje działanie

In [15]:
print gen == iter(gen)
next(gen)

True


StopIteration: 

Inny sposób sposób tworzenia iteratora (ale nie generatora):

In [19]:
class myiter(object):
    def __init__(self, v):
        self.v = v
    def __iter__(self):
        return self
    def next(self):
        self.v = self.v**2
        if self.v > 1000000000:
            raise StopIteration()
        return self.v
    
for i in myiter(2):
    print i

4
16
256
65536


Generatory używa się ich wraz z pętlami for (tak samo jak zwykłe iteratory):

In [17]:
def next_cubes(n):
    i = 1
    while i <= n:
        yield i**3
        i += 1

for i in next_cubes(5):
    print i

1
8
27
64
125


Po co?

In [18]:
import sys
print sys.getsizeof(xrange(100))
print sys.getsizeof(range(100))

40
872


In [23]:
def next_cubes2(n):
    for i in range(1, n):
        yield i**3

python gen_while_check.py

11.5625

python gen_range_check.py

42.54296875

## 4.1. Wyrażenia generatorowe

In [23]:
results = (a**2 for a in xrange(100))
print results

<generator object <genexpr> at 0x7fa4e57132d0>


## 4.2. Użycie

Nieskończone sekwencje i sekwencje o nieznanej długości.

In [28]:
def fib():
    a, b = 1, 1
    yield b
    yield a
    while True:
        a, b = a + b, a
        yield a

for i in fib():
    if i >= 5:
        break
    print i

1
1
2
3


In [33]:
def dictwalk(items):
    stack = [items]
    while stack:
        it = stack.pop()
        for key in it:
            if type(it[key]) is dict:
                stack.append(it[key])
            else:
                yield key, it[key]
                
                
dct = {"a": 1, "b": {"1a": 11, "1b": 15, "1c": {"11a": 102}},
       "c": 4, "d": {"2a": 12, "2b": {"22a": 150}}}
for k, v in dictwalk(dct):
    print k, v
    

<generator object walk at 0x7fb5800f18c0>


Operacje na plikach i katalogach.

In [20]:
import os
print os.walk('.')

<generator object walk at 0x7fa4dc200d70>


In [27]:
with open('./long_log') as log:
    result = (int('error' in line) for line in log)
    print sum(result)

1


Cooperative multitasking. Corutines.

In [17]:
import time
from collections import deque


class Result(object):
    def __init__(self, v):
        self.v = v
    def __repr__(self):
        return "<Result: {}>".format(self.v)

        
def scheduler(jobs):
    results = []
    not_finished = deque(jobs)
    while not_finished:
        try:
            job = not_finished.pop()
            result, name = next(job)
            if result:
                results.append(result)
            else:
                print "Switched from: {}".format(name)
                not_finished.appendleft(job)
        except StopIteration:
            pass
    return results


def job(name, n, a=3):
    val = 1
    for i in xrange(n-1):
        val += a * val  + 1
        time.sleep(0.3)
        yield None, name
    yield Result(a * val + 1), name

        
print scheduler([job("fst", 5), job("sec", 6, 10), job("thr", 2, 100)])


Switched from: thr
Switched from: sec
Switched from: fst
Switched from: sec
Switched from: fst
Switched from: sec
Switched from: fst
Switched from: sec
Switched from: fst
Switched from: sec
[<Result: 10201>, <Result: 1024>, <Result: 1771561>]


## Zadanie

Napisz generator, który zwróci wszystkie numery linii z pliku, w których (liniach) występuje słowo 'codilime'.

In [31]:
def get(word='codilime'):
    with open('./long_log'):
        pass

## 4.3. Podsumowanie

* Generatory zachowują stan pomiędzy kolejnymi wywołaniami
* Generatory to iteratory
* Ich użycie wiąże się z mniejszym wykorzystaniem dostępnej pamięci
* Mają wiele zastosowań: przeszukiwanie, nieskończone sekwencje, pipeline'y, lekkie wątki

## 4.4. Czego nie było

* metody send
* bardziej rozbudowanych przykładów współbieżności z użyciem generatorów

http://www.dabeaz.com/coroutines/